In [1]:
!pip install nbimporter

In [1]:
!pip install pydub

In [69]:
!pip install soundfile --upgrade

In [4]:
!pip install librosa

   ---------------------------------------- 0.0/260.1 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/260.1 kB 1.3 MB/s eta 0:00:01
   ---- ----------------------------------- 30.7/260.1 kB 1.3 MB/s eta 0:00:01
   ------------- ------------------------- 92.2/260.1 kB 655.4 kB/s eta 0:00:01
   ---------------- --------------------- 112.6/260.1 kB 652.2 kB/s eta 0:00:01
   ---------------------------- --------- 194.6/260.1 kB 841.6 kB/s eta 0:00:01
   ---------------------------------- --- 235.5/260.1 kB 901.1 kB/s eta 0:00:01
   -------------------------------------- 260.1/260.1 kB 841.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/75.3 kB ? eta -:--:--
   ---------------------------------------- 75.3/75.3 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.6 kB ? eta -:--:--
   ---------------------------------------- 64.6/64.6 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---

In [1]:
vulnerable_species = [
    "Chloropsis jerdoni",    # Jerdon's Chloropsis
    "Clanga hastata"         # Indian Spotted Eagle
]

In [2]:
near_threatened_species = [
    "Dicaeum cruentatum",    # Scarlet-backed Flowerpecker
    "Glaucidium cuculoides", # Asian Barred Owlet
    "Halcyon coromanda",     # Ruddy Kingfisher
    "Ixobrychus cinnamomeus",# Cinnamon Bittern
    "Liocichla phoenicea",   # Red-faced Liocichla
    "Loriculus vernalis",    # Vernal Hanging Parrot
    "Macronus gularis",      # Pin-striped Tit-Babbler
    "Butorides striata",     # Striated Heron
    "Chrysococcyx maculatus" # Asian Emerald Cuckoo
]


In [3]:
normal = [
    "Acridotheres fuscus",
    "Acrocephalus bistrigiceps",
    "Aethopyga gouldiae",
    "Alcippe cinerea",
    "Arachnothera magna",
    "Arborophila torqueola",
    "Argya longirostris",
    "Centropus andamanensis",
    "Chelidorhynx hypoxanthus",
    "Chloropsis",
    "Chloropsis cochinchinensis",
    "Cuculus micropterus",
    "Cyornis concretus",
    "Cyornis magnirostris",
    "Cyornis poliogenys",
    "Cyornis unicolor",
    "Cypsiurus balasiensis",
    "Dicaeum chrysorrheum",
    "Dicaeum minullum",
    "Dicrurus andamanensis",
    "Eurystomus orientalis",
    "Accipiter badius",
    "Accipiter nisus",
    "Amaurornis phoenicurus",
    "Anastomus oscitans",
    "Anser anser",
    "Apus affinis",
    "Arborophila torqueola",
    "Butorides striata",
    "Cacomantis merulinus",
    "Cacomantis passerinus",
    "Cacomantis sonneratii",
    "Caprimulgus affinis",
    "Caprimulgus asiaticus",
    "Caprimulgus atripennis",
    "Caprimulgus macrurus",
    "Centropus sinensis",
    "Clamator jacobinus",
    "Columba livia"
]

In [34]:
!pip install librosa matplotlib numpy

Note: you may need to restart the kernel to use updated packages.


In [6]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.transform import resize
import uuid
from PIL import Image
import tempfile
from soundfile import LibsndfileError
import os
import matplotlib.pyplot as plt
from scipy.signal import sosfilt, butter

In [14]:
def audio_augmentation(y , sr , save_directory):
    try:
        y_stretched = librosa.effects.time_stretch(y, rate=1.2)  # Speed up by 20%
        makespectogram(y_stretched , sr , save_directory)
        y_shifted = librosa.effects.pitch_shift(y, sr = sr, n_steps=2)
        makespectogram(y_shifted , sr , save_directory)
        noise = np.random.normal(0, 0.05, y.shape)
        y_noisy = y + noise
        makespectogram(y_noisy , sr , save_directory)
        y_louder = y * 1.5  # Increase volume by 50%
        makespectogram(y_louder , sr , save_directory)
        sos = butter(10, 1000, 'hp', fs=sr, output='sos')  # High-pass filter
        y_filtered = sosfilt(sos, y)
        makespectogram(y_filtered , sr , save_directory)
    except (LibsndfileError, Exception) as e:
        print(f"Error processing audio data: {e}")

In [8]:
# Spectrogram Augmentation
def spec_augment(spec, max_time_warp=5, max_freq_mask=2, max_time_mask=2):
    try:
        num_freq_bins, num_time_steps = spec.shape
        time_warp = np.random.randint(-max_time_warp, max_time_warp)
        spec = np.roll(spec, time_warp, axis=1)
        freq_mask = np.random.randint(0, max_freq_mask)
        spec[np.random.randint(0, num_freq_bins - freq_mask): np.random.randint(0, num_freq_bins - freq_mask) + freq_mask, :] = 0
        time_mask = np.random.randint(0, max_time_mask)
        spec[:, np.random.randint(0, num_time_steps - time_mask): np.random.randint(0, num_time_steps - time_mask) + time_mask] = 0
        return spec
    except (LibsndfileError, Exception) as e:
        print(f"Error processing audio data: {e}")
        

In [9]:
def makespectogram(audio_data, sr, save_directory):
    try:
        # Define parameters
        n_fft = 512  # Window length: 256 or 512
        hop_length = 128  # Hop size: 64 or 128

        # Compute the STFT
        D = librosa.stft(audio_data, n_fft=n_fft, hop_length=hop_length)
        S = np.abs(D)
        S_db = librosa.amplitude_to_db(S, ref=np.max)

        # Define the number of Mel bands
        n_mels = 128

        # Compute the Mel spectrogram
        S_mel = librosa.feature.melspectrogram(S=S, sr=sr, n_mels=n_mels)
        spec_augmented = spec_augment(S_mel)

        lst = [S_mel , spec_augmented]

        for S_mel in lst:
            
            S_mel_db = librosa.amplitude_to_db(S_mel, ref=np.max)
    
            # Define the target size
            target_size = (128, 512)  # Can be [128, 512], [96, 512], or [128, 1024]
    
            # Rescale the Mel spectrogram
            S_rescaled = resize(S_mel_db, target_size, mode='reflect', anti_aliasing=True)
    
            # Normalize the data to range [0, 255]
            S_rescaled_normalized = 255 * (S_rescaled - np.min(S_rescaled)) / (np.max(S_rescaled) - np.min(S_rescaled))
            S_rescaled_normalized = np.stack((S_rescaled_normalized,)*3, axis=-1)
            S_rescaled_normalized = S_rescaled_normalized.astype(np.uint8)
    
            # Save the spectrogram as an image file using Pillow
            image = Image.fromarray(S_rescaled_normalized)
            image.save(os.path.join(save_directory, '{}.png'.format(uuid.uuid1())))  # Save the image
    
            return plt
    except (LibsndfileError, Exception) as e:
        print(f"Error processing audio data: {e}")

In [10]:
def split_audio_librosa(audio_file, save_directory, chunk_duration=2500):

    # Check if the file exists
    if not os.path.exists(audio_file):
        print(f"File not found: {audio_file}")
        return

    try:
        # Load audio
        y, sr = librosa.load(audio_file)
    except Exception as e:
        print(f"Error loading audio file: {e}")
        return

    # Calculate number of samples per chunk
    chunk_size = int(chunk_duration * sr / 1000)
    try:
        for i in range(0, len(y), chunk_size):
            chunk = y[i:i+chunk_size]
            audio_augmentation(y , sr , save_directory)
            # makespectogram(chunk, sr, save_directory)
    except (LibsndfileError, Exception) as e:
        print(f"Error processing audio data: {e}")
        

In [11]:
folder = ['normal' , 'endangered' , 'vulnerable']

In [8]:
data = pd.read_csv('./archive/train_extended.csv')

In [9]:
species_lst = data['species'].unique()

In [10]:
rare_birds = [
    'American Bittern', 
    'Black-billed Cuckoo', 
    'Blackburnian Warbler', 
    "Baird's Sandpiper", 
    'Broad-tailed Hummingbird', 
    'Blackpoll Warbler', 
    "Bell's Sparrow", 
    'Ash-throated Flycatcher', 
    'Broad-winged Hawk', 
    'Sand Martin', 
    'Greater Roadrunner', 
    "LeConte's Thrasher", 
    'Long-billed Curlew', 
    'Mangrove Warbler', 
    'Rusty Blackbird'
]


In [11]:
species_lst

array(['Alder Flycatcher', 'American Avocet', 'American Bittern',
       'American Crow', 'American Goldfinch', 'American Kestrel',
       'Buff-bellied Pipit', 'American Redstart', 'American Robin',
       'American Wigeon', 'American Woodcock', 'American Tree Sparrow',
       "Anna's Hummingbird", 'Ash-throated Flycatcher',
       "Baird's Sandpiper", 'Bald Eagle', 'Baltimore Oriole',
       'Sand Martin', 'Barn Swallow', 'Black-and-white Warbler',
       'Belted Kingfisher', "Bell's Sparrow", "Bewick's Wren",
       'Black-billed Cuckoo', 'Black-billed Magpie',
       'Blackburnian Warbler', 'Black-capped Chickadee',
       'Black-chinned Hummingbird', 'Black-headed Grosbeak',
       'Blackpoll Warbler', 'Black-throated Sparrow', 'Black Phoebe',
       'Blue Grosbeak', 'Blue Jay', 'Brown-headed Cowbird', 'Bobolink',
       "Bonaparte's Gull", 'Barred Owl', "Brewer's Blackbird",
       "Brewer's Sparrow", 'Brown Creeper', 'Brown Thrasher',
       'Broad-tailed Hummingbird', 'Broad-wi

In [12]:
filter_data = data[data['species'].isin(rare_birds)]

In [13]:
pick = filter_data['ebird_code'].unique()
pick = pick.tolist()
len(pick)

15

In [14]:
import shutil

In [ ]:
for i in os.listdir('.//archive/N-Z'):
    if i in pick :
        shutil.move(os.path.join('.//archive/N-Z' , i) , './vulnerable/')

In [12]:
import warnings
import glob
warnings.filterwarnings('ignore')

for i in folder:
    for j in os.listdir(i):
        for k in os.listdir(os.path.join(i , j)):
            png_files = glob.glob(os.path.join(i,j, "*.png"))
             # Loop through the list and delete each file
            for file in png_files:
                try:
                    os.remove(file)
                    print(f"Deleted: {file}")
                except Exception as e:
                    print(f"Error deleting {file}: {e}")
            # path = os.path.join(i,j,k)
            # save_path = os.path.join(i,j)
            # makespectogram(path , save_path)
            
            

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for i in folder:
    for j in os.listdir(i):
        save_path = os.path.join(i,j)
        for k in os.listdir(os.path.join(i , j)):        
            path = os.path.join(i,j,k)
            split_audio_librosa(path , save_path)
        print("first sucess")

Error loading audio file: 
Error loading audio file: 
Error loading audio file: 
Error loading audio file: 
Error loading audio file: 
Error loading audio file: 
Error loading audio file: 
Error loading audio file: 
Error loading audio file: 


In [17]:
final_name_list = []
for directory in folder:
    for sub_dir in os.listdir(directory):
        png_files = glob.glob(os.path.join(directory,sub_dir, "*.png"))
        # if int(len([name for name in os.listdir(os.path.join(directory , sub_dir)) if os.path.isfile(os.path.join(directory , sub_dir, name))])/2) > 20:
        if int(len(png_files)) > 1000 :
            print(sub_dir , " " , int(len(png_files)))
            final_name_list.append(sub_dir)
        else :
            shutil.move(os.path.join(directory , sub_dir) , os.path.join("extra"))
    print()

Accipiter_badius normal normal   1655
Amaurornis_phoenicurus normal normal   1655
Anastomus_oscitans normal normal   1655
Anser_anser normal normal   1655
Apus_affinis normal normal   1655
Arborophila_torqueola normal normal   1655
Butorides_striata normal   1655
Cacomantis_merulinus normal   1655
Cacomantis_passerinus normal   1655
Cacomantis_sonneratii normal   1655
Caprimulgus_affinis normal   1655
Caprimulgus_asiaticus normal   1655
Caprimulgus_atripennis normal   1655
Centropus_sinensis normal   1655
Chrysococcyx_maculatus normal   1655
Clamator_jacobinus normal   1655
Columba_livia normal   1655

Glaucidium cuculoides endangered   1398
Liocichla phoenicea endangered   1001

Ash-throated Flycatcher vulnerable   10022
Blackburnian Warbler vulnerable   1146
Blackpoll Warbler vulnerable   2762
Clanga_hastata vulnerable   1655
Mangrove Warbler vulnerable   1363
Rusty Blackbird vulnerable   1194
Sand Martin vulnerable   2504

